<a href="https://colab.research.google.com/github/kirnap/learnbyfun/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  25.2M      0  0:00:03  0:00:03 --:--:-- 25.2M


In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [5]:
!rm -r aclImdb/train/unsup

In [6]:
!head aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [7]:
batch_size=32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=1337
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=1337,
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size
)
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [8]:
# This is to take a look at the raw data to ensure normalization, and tokenization work as expected
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

### Prepare the data
---
In particular remove `<br />` tags

In [9]:
from tensorflow.keras.layers import TextVectorization
import string
import re


def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
  return tf.strings.regex_replace(
      stripped_html, f"[{re.escape(string.punctuation)}]", ""
  )

# Model constants
max_features = 20000
embedding_dim = 128
sequence_length = 500


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)


text_ds = raw_train_ds.map(lambda x,y: x)  # text-only dataset(no label)
# Call `adapt` on a text-only dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer.adapt(text_ds)

In [10]:
# There are 2 ways we can use our text vectorizeation layer:
# Option1: Make it part of the model, so as to obtain a model that processes raw strings, like this:

# text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
# x = vectorize_layer(text_input)
# x = layers.Embedding(max_features+1, embedding_dim)(x)

**Option 2: Applying vectorization layer to text dataset**  to obtain a dataset of word indices, then feed it into a model that expects integer sequences as inputs.

This option enables asynchronous CPU processing and buffering of the data.

To export model to production, we'd ship a model that accepts raw strings as input, like in the code snippet for option 1 above. This can be done after training. We do this in the last section.

In [11]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

# data vectorization
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)


# async prefetching / buffering of the data for the best performance on a GPU
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10) 


# Build a model

In [12]:
from tensorflow.keras import layers

# An integer input for vocab indices
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, add a layer to map vocab indices into a shape of dimensionality
# 'embedding_dim'
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)


# conv1d + global max pooling
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.Conv1D(128, 7, padding='valid', activation='relu', strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# add a vanilla hidden layer:
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# project onto a single unit output layer, and squash it with a sigmoid
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = tf.keras.Model(inputs, predictions)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
# model training
epochs = 10
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
625/625 [==============================] - 11s 18ms/step - loss: 0.0685 - accuracy: 0.9764 - val_loss: 0.4919 - val_accuracy: 0.8658
Epoch 2/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0441 - accuracy: 0.9853 - val_loss: 0.5455 - val_accuracy: 0.8730
Epoch 3/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0352 - accuracy: 0.9883 - val_loss: 0.5606 - val_accuracy: 0.8656
Epoch 4/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0214 - accuracy: 0.9930 - val_loss: 0.7995 - val_accuracy: 0.8598
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0180 - accuracy: 0.9939 - val_loss: 0.8202 - val_accuracy: 0.8612
Epoch 6/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0137 - accuracy: 0.9951 - val_loss: 0.8791 - val_accuracy: 0.8622
Epoch 7/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0136 - accuracy: 0.9951 - val_loss: 0.8316 - val_accuracy:

In [16]:
# This is to evaluate test accuracy of the model
model.evaluate(test_ds)

782/782 [==============================] - 4s 5ms/step - loss: 0.9122 - accuracy: 0.8599


[0.9122275114059448, 0.8599200248718262]